In [122]:
import numpy as np
import pandas as pd

import itertools
import math

import matplotlib.pyplot as plt
from matplotlib import gridspec

from tqdm.notebook import tqdm
import pymorphy2

import nltk
from nltk.tokenize import RegexpTokenizer

from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel

from sklearn.feature_extraction.text import CountVectorizer

import tqdm

from pymystem3 import Mystem

from Levenshtein import distance as lev

import yaml
nltk.download('stopwords')
# pymorphy2 анализатор
morph = pymorphy2.MorphAnalyzer()
# стоп слова из nltk
stops = nltk.corpus.stopwords.words('russian')

%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package stopwords to /home/art-
[nltk_data]     bash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [123]:
with open('topics.yaml', 'r') as file:
    topics_signature = yaml.safe_load(file)

In [124]:
FL = '../dat/glue_all_but_owner.txt'

topics_signature

{'QUESTION': [['почем'], ['подскажите'], ['есть'], ['условия'], ['цена']],
 'BUY': [['хочу', 'купить'], ['рассрочк'], ['сколько', 'стоит'], ['подарок']],
 'REVIEW': [['спасибо']],
 'INSTALLMENT': [['Грозн'], ['рассрочк']],
 'ADVERTISEMENT': [['реклама'], ['сотрудничеств']],
 'APPLE': [['iphone'], ['айфон'], ['apple'], ['эппл'], ['вотч'], ['watch']],
 'HEADPHONES': [['наушник'], ['airpods']],
 'DELIVERY': [['доставка']],
 'ANDROID': [['самсунг'], ['ксиаоми'], ['xiaomi'], ['mi']]}

In [132]:
import pymorphy2

import nltk
from nltk.tokenize import RegexpTokenizer


import tqdm


from Levenshtein import distance as lev

import yaml
nltk.download('stopwords')
# pymorphy2 анализатор
morph = pymorphy2.MorphAnalyzer()
# стоп слова из nltk
stops = nltk.corpus.stopwords.words('russian')

with open('topics.yaml', 'r') as file:
    topics_signature = yaml.safe_load(file)

def only_alpha_from_string(x):
    return ''.join(a for a in x if (a.isalpha() or a == ' '))


def get_min_dist(msg, tag):
    min_lev = 1e6
    for word in msg:
        min_lev = min(lev(word, tag), min_lev)
    return min_lev


def check_tag(message_text, tag_content, lev_threshold=3):
    fail_flg = False
    for tag in tag_content:
        if get_min_dist(message_text, tag) > lev_threshold:
            fail_flg = True
            break
    return not fail_flg

def check_topic(message_text, tags):
    has_topic = False
    relevant_tags = []
    for tag in tags:
        if check_tag(message_text, tag):
            has_topic = True
            relevant_tags.append(tag)
    return has_topic, relevant_tags


def check_message(orig_message_text, config):
    message_text = only_alpha_from_string(orig_message_text).lower().split()
    message_text = [x for x in [morph.normal_forms(word)[0] for word in message_text] \
             if x not in stops]
    #print(message_text)
    result = {}
    for topic_name in config:
        check_result, tags = check_topic(message_text, config[topic_name])
        if check_result:
            result[topic_name] = tags

    result_message = ''
    for fin_top in result:
        result_message += '<' + fin_top + '> '
    result_message += orig_message_text

    return result_message

check_message('хочу купить айфон в рассрочку', topics_signature)

[nltk_data] Downloading package stopwords to /home/art-
[nltk_data]     bash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'<BUY> <INSTALLMENT> <APPLE> хочу купить айфон в рассрочку'

In [126]:
with open(FL, 'r') as fl:
    messages = fl.readlines()
    messages = [m.replace('\n', '') for m in messages]


In [127]:
ehehe = ['сколько стоят Наушники в рассрочку?']

In [128]:
topic_separation = []

for ms in tqdm.tqdm(messages[:100]):
    topic_separation.append(check_message(ms, topics_signature))

100%|██████████| 100/100 [00:00<00:00, 701.25it/s]


In [129]:
topic_separation

['<QUESTION> <APPLE> сегодня смотреть , цвет , дело , мочь дать знать , цвет вариант , цена брат , время , цвет наличие , скидка мочь , брат мочь отдать , суббота сестра , брат , реквизит деньга часы , дать дойла , брат наличие , вариант , завтра , цена отдать ',
 '<APPLE> айфон мяло брат , взнос мяло вариант ',
 '<QUESTION> день цена , день цена ',
 '<QUESTION> <BUY> <INSTALLMENT> <APPLE> андроид камера мочь , каков условие рассрочка ',
 '<QUESTION> <BUY> <INSTALLMENT> <APPLE> айфон , белый , белый , цвет значение , взнос , рассрочка месяц , карантин ',
 '<QUESTION> <BUY> <INSTALLMENT> <APPLE> iphone , наличие , цена , iphone красный цена , условие рассрочка , наличие , цена ',
 '<QUESTION> <INSTALLMENT> <APPLE> часы , знать , айфон , айфон , защита , бронь , обмен , дать видеть телефон , царапина ',
 '<QUESTION> <BUY> <INSTALLMENT> брат , условие рассрочка , справка работа ',
 '<QUESTION> <APPLE> вечер , цена белый айфон ',
 '<QUESTION> <INSTALLMENT> <APPLE> бронь , дело , айфон голд

In [130]:
lev('вотч', 'стоят')

4